# 1. Introducción a los datos

## Contenido
---

**Introducción**
- ¿Qué es la detección de noticias falsas? (*Fake news*).
- Objetivo de la libreta.

**Información de los datos**
- Información general.
- Temas de las noticias.
- Estructura de los datos.

**Análisis de los datos**
- Librerías.
- Cargar los datos.
- Información de los datos.
- ¿Cuáles son las fuentes?
- Eliminar fuentes repetidas.
- Procesar todas las fuentes.
- La importancia de limpiar los datos.

## Introducción
---

#### ¿Qué es la detección de *Fake News*?

La detección de *Fake News* (noticias falsas) es la tarea de evaluar la veracidad de las afirmaciones en las noticias. Este es un problema crítico en el Procesamiento del Lenguaje Natural (PLN) porque tanto en medios de noticias tradicionales como en medios digitales las *Fake News* generan un gran impacto social y político en cada individuo. Por ejemplo, la exposición a las *Fake News* puede generar actitudes de ineficacia, alienación y cinismo hacia ciertos candidatos políticos (Balmas, 2014).

#### Objetivo de la libreta

El objetivo de esta libreta es proporcionar un análisis de los datos, su estructura e información básica.

## Información de los datos
---

#### Información general

Los datos son una recopilación de noticias extraídas de distintas fuentes por el usuario en Github [pipe11](https://github.com/pipe11/TFM_fake_news_detector),
los datos contienen en total 3974 artículos en español, distribuidos en 2046 *True News* y 1918 *Fake News*. Utilizando la estructura propuesta por [J.P. Posadas](https://github.com/jpposadas/FakeNewsCorpusSpanish).

Las fuentes originales de los datos son:
1. [jpposadas - FakeNewsCorpusSpanish](https://github.com/jpposadas/FakeNewsCorpusSpanish): 971 noticias.
2. [several127 - FakeNewsCorpus](https://github.com/several27/FakeNewsCorpus): 9,408,908 artículos, la mayoría en inglés.
3. [WebHouse Dataset](https://webz.io/free-datasets/spanish-news-articles/): 342,000 artículos en español.

Para obtener el conjunto de datos final se requirió:
1. Filtrar todas las noticias en español de la fuente 2.
2. Clasificar la noticia según su tema.
3. Clasificar las noticias como *True* o *Fake* usando un clasificador basado en los datos de J.P. Posadas.

#### Temas de las noticias

El *corpus* cubre noticias de 9 temas diferentes.

1. Ciencia.
2. Deportes.
3. Economía.
4. Entretenimiento.
5. Educación.
6. Política.
7. Salud.
8. Seguridad.
9. Sociedad

#### Estructura de los datos

- Id: Identificador de cada instancia.
- Category: Categoría de la noticia (True o False).
- Topic: Tema relacionado con la noticia.
- Source: Nombre de la fuente.
- Headline: Titulo de la noticia.
- Text: Texto de la noticia	.
- Link: URL de la fuente.


## Análisis de los datos
---

#### Librerías

In [1]:
import pandas as pd
import cmd
import unidecode
import re

#### Cargar los datos

In [2]:
corpus = pd.read_csv('Datos/corpus_spanish_v3.csv', index_col=0)
corpus.head()

,Category,Topic,Source,Headline,Text,Link
0,True,Science,El país,La NASA recupera el contacto con un satélite d...,La NASA recupera el contacto con un satélite d...,https://elpais.com/elpais/2018/02/01/ciencia/1...
1,True,Economy,El financiero,AMLO aceleraría el consumo y el crecimiento ec...,AMLO aceleraría el consumo y el crecimiento ec...,http://www.elfinanciero.com.mx/economia/amlo-a...
2,Fake,Sport,actualidadpanamericana.com,Compañero de James se ‘calvea’ y le juega pesa...,Al borde de un colapso nervioso quedó el hábil...,http://www.actualidadpanamericana.com/companer...
3,Fake,Politics,actualidadpanamericana.com,Dian gravará este año a los niños que recojan ...,"Mediante pupitrazo de último minuto anoche, el...",http://www.actualidadpanamericana.com/dian-gra...
4,Fake,Politics,actualidadpanamericana.com,Uribe asegura que insultó 358 guerrilleros,Muy temprano esta mañana el expresidente y aho...,http://www.actualidadpanamericana.com/uribe-as...


#### Información de los datos

In [3]:
print(f'News, Columns: {corpus.shape}')
print('True News: {}'.format(corpus.groupby('Category').size()[0]))
print('Fake News: {}'    .format(corpus.groupby('Category').size()[1]))
print(f'Columns Name: {list(corpus.columns)}')

News, Columns: (3974, 6)
True News: 1928
Fake News: 2046
Columns Name: ['Category', 'Topic', 'Source', 'Headline', 'Text', 'Link']


#### ¿Cuáles son las fuentes?

Los datos no siempre se encuentran limpios y ordenados, a veces es necesario procesar los datos antes de poder aplicar un método que solucione el problema, el obtener una lista de todas las fuentes parece ser una tarea sencilla, sin embargo, no lo es.

Para intentar obtener las fuentes se realizan estos pasos:
1. Obtener todos los valores de la columna *Source*.
2. Eliminar todas las fuentes repetidas.
3. Imprimir las primeras 15 fuentes.

In [4]:
# 1.
sources_ls = corpus['Source'].to_list()

# 2.
sources_ls = list(sorted(set(sources_ls)))
print(f'Sources: {len(sources_ls)}')

# 3.
cli = cmd.Cmd()
cli.columnize(sources_ls[:15], displaywidth=130)

Sources: 182
20 Minutos     ABC Noticias       ARGUMENTO POLÍTICO  Animal Politico   Argumento Político
ABC            ABC sociedad       Actualidad RT       Animal Político   Aristegui Noticias
ABC EL RECREO  ANONYMOUS  AZTECA  Alerta digital      Animal político   Aristegui noticias


#### Eliminar fuentes repetidas

Se puede observar que existen fuentes repetidas en la lista como 'Animal Político' o 'Aristegui Noticias' porque no se escribieron de la misma manera, sin embargo, son la misma fuente de información. Para resolver este problema es necesario crear una función que procese el texto de las fuentes.

La función que nos permite procesar el texto sigue estas reglas:
1. Convierte todo el texto en minúsculas.
2. Elimina los acentos de las palabras.
3. Elimina todos los símbolos que no son letras, para obtener solo las palabras del texto.

In [5]:
def process_sources(source_text):
    text = source_text
    text = text.lower()
    text = re.sub(r'(https?://)?(www\.)?(wiki\.)?(in\.)?(www2\.)?(news\.)?', "", text)
    text = re.sub(r'(\.blogspot\.com)|(.com)|(\.org)|(\.es)|(\.fo)', "", text)
    text = re.sub(r'http', "", text)
    text = re.sub(r'@\S+', "", text)
    text = re.sub(r'\n', "", text)
    text = re.sub(r'/', "", text)
    text = re.sub(r'^[ \t]+|[ \t]+$', "", text)
    text = re.sub(r"(?<!\n)\n(?!\n)", " ", text)
    text = re.sub(r'[ -]+', " ", text)
    text = re.sub(r' +', " ", text)
    text = text.replace(r"*PHONE*", "número")
    text = text.replace(r"*EMAIL*", "email")
    text = text.replace(r"*URL*", "url")
    text = unidecode.unidecode(text)

    return text

#### Procesar todas las fuentes

Con la función `process_sources` y utilizando el método `apply()` de la estructura `DataFrame`, se aplica la función a cada uno de las fuentes del conjunto de datos, después se obtiene la lista de las fuentes como se hizo anteriormente, pero ahora se imprimirán todas las fuentes.

Para obtener la lista de fuentes real se realizan los siguientes pasos:
1. Aplicar la función `process_sources`.
2. Obtener todos los valores de la columna *Source*.
3. Eliminar todas las fuentes repetidas.
4. Imprimir las primeras 15 fuentes.

In [6]:
# 1.
processed_sources = corpus['Source'].apply(process_sources)

# 2.
processed_sources_ls = processed_sources.to_list()

# 3.
processed_sources_set = set(processed_sources_ls)
processed_sources_ls = list(sorted(processed_sources_set))
print(f'Sources: {len(processed_sources_ls)}')

# 4.
cli.columnize(processed_sources_ls, displaywidth=130)

Sources: 162
20 minutos              diario de ibiza            exclusivas puebla        la voz popular         radio formula          
abc                     diario del huila           expansion                lamula                 razon                  
abc el recreo           dos mundos                 facebook el senor lopez  latino                 record                 
abc noticias            eco portal                 facebook unico noticias  lo que calla la tv     resumen latinoamericano
abc sociedad            eje central                fifa                     los replicantes        retroceso              
actualidad rt           el buen tono               fisgon politico          marca                  reuters                
actualidadpanamericana  el clarin                  forbes                   medio tiempo           rey misterios          
alerta digital          el coleccionista de datos  frentes politicos        mediterraneo digital   sabor a merengue       
ani

#### La importancia de limpiar los datos

La primera vez se obtuvieron 182 fuentes en total, después de ser procesadas se obtuvieron 162 fuentes. Este resultado resalta la importancia del procesado del texto, sin este proceso, se contemplarían 20 fuentes de información adicionales, el mismo resultado puede ser obtenido al procesar todo el texto de la noticia para evitar palabras repetidas o que pierdan importancia por encontrarse mal escritas.